# Importing All the Necessary Libraries and Having A Overlook on Data

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sevilla-football/sample_submission_wBWLI0s.csv
/kaggle/input/sevilla-football/train_1K0BDt5/train.csv
/kaggle/input/sevilla-football/test_kuhCxHY/test.csv


In [2]:
train_path = '../input/sevilla-football/train_1K0BDt5/train.csv'
test_path = '../input/sevilla-football/test_kuhCxHY/test.csv'
sub_path = '../input/sevilla-football/sample_submission_wBWLI0s.csv'

In [3]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
df_sub = pd.read_csv(sub_path)

### Dropping Row Id and Rating Num before combining train and test data to do preprocessing at one go 

In [4]:
df_combo = pd.concat([df_train.drop(['rating_num','row_id'],axis=1),df_test],ignore_index = False)
df_combo

,scout_id,winner,team,competitionId,player_position_1,player_position_2,player_height,player_weight,player_general_var_0,player_general_var_1,...,team2_offensive_derived_var_11,team2_offensive_derived_var_12,team2_offensive_derived_var_13,team2_offensive_derived_var_14,team2_defensive_derived_var_15,team2_offensive_derived_var_16,team2_defensive_derived_var_17,team2_offensive_derived_var_18,team2_offensive_derived_var_19,row_id
0,13,winner,team1,8.0,7.0,7.0,0.317073,0.48,0.0,0,...,0.131548,0.097131,0.600944,0.338892,0.347980,0.034157,0.288962,0.310727,0.204925,NaN
1,16,loser,team2,8.0,3.0,9.0,0.463415,0.42,0.0,0,...,0.173697,0.272801,0.605659,0.526761,0.468976,0.054034,0.334962,0.380073,0.295392,NaN
2,4,loser,team1,5.0,11.0,11.0,0.682927,0.44,0.0,0,...,0.250756,0.212631,0.558833,0.686523,0.441749,0.219441,0.414288,0.570757,0.437500,NaN
3,13,loser,team2,4.0,17.0,17.0,0.682927,0.58,0.0,0,...,0.155969,0.206503,0.533002,0.373959,0.527733,0.095201,0.423040,0.330754,0.235558,NaN
4,11,draw,team1,5.0,10.0,10.0,0.731707,0.70,0.0,0,...,0.199050,0.134002,0.688773,0.681724,0.733625,0.128719,0.448360,0.444946,0.312500,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8769,15,loser,team1,7.0,11.0,11.0,0.463415,0.60,0.0,0,...,0.290421,0.342837,0.572133,0.721060,0.683173,0.341016,0.400924,0.554274,0.275819,21259.0
8770,2,draw,team1,4.0,16.0,16.0,0.341463,0.40,0.2,0,...,0.156339,0.173252,0.803287,0.449554,0.443569,0.087641,0.423432,0.435869,0.141724,9371.0
8771,2,loser,team2,4.0,1.0,1.0,0.439024,0.36,0.0,0,...,0.182449,0.306235,0.591016,0.484777,0.804573,0.094133,0.250106,0.358223,0.376893,23226.0
8772,11,winner,team1,10.0,1.0,1.0,0.463415,0.54,0.0,0,...,0.141524,0.230742,0.609988,0.356299,0.334398,0.045464,0.429783,0.503833,0.091779,26623.0


# DATA PRE-PROCESSING 

### We have dropped all the columns which have 100% NaN Values

In [5]:
for column in df_combo.select_dtypes('float64').columns.to_list():
    if df_combo[column].isnull().sum()/len(df_combo) == 1:
        df_combo = df_combo.drop(column,axis=1)

### We have adopted simple mean value imputation

In [6]:
for column in df_combo.select_dtypes('float64').columns.to_list():
    if df_combo[column].isnull().sum() != 0:
        df_combo[column] = df_combo[column].fillna(df_combo[column].mean())

### From Here on we have applied Quantile Transformer on selected few columns which mainly come under the group of 'Derived' and 'Ratio' Columns to make their distribution normal which would be good for modelling

In [7]:
from sklearn.preprocessing import QuantileTransformer
l1 = [x for x in df_combo.columns.to_list() if x.startswith('player_general_var')]
for column in l1:
    qt1 = QuantileTransformer(output_distribution = 'normal')
    df_combo[column] = qt1.fit_transform(df_combo[column].values.reshape(-1,1))


In [8]:
from sklearn.preprocessing import QuantileTransformer
l1 = [x for x in df_combo.columns.to_list() if x.startswith('player_general_derived_var')]
for column in l1:
    qt2 = QuantileTransformer(output_distribution = 'normal')
    df_combo[column] = qt2.fit_transform(df_combo[column].values.reshape(-1,1))


In [9]:
from sklearn.preprocessing import QuantileTransformer
l1 = [x for x in df_combo.columns.to_list() if x.startswith('player_defensive_ratio_var')]
for column in l1:
    qt3 = QuantileTransformer(output_distribution = 'normal')
    df_combo[column] = qt3.fit_transform(df_combo[column].values.reshape(-1,1))


In [10]:
from sklearn.preprocessing import QuantileTransformer
l1 = [x for x in df_combo.columns.to_list() if x.startswith('player_defensive_derived_var')]
for column in l1:
    qt4 = QuantileTransformer(output_distribution = 'normal')
    df_combo[column] = qt4.fit_transform(df_combo[column].values.reshape(-1,1))


In [11]:
from sklearn.preprocessing import QuantileTransformer
l1 = [x for x in df_combo.columns.to_list() if x.startswith('player_offensive_derived_var')]
for column in l1:
    qt5 = QuantileTransformer(output_distribution = 'normal')
    df_combo[column] = qt5.fit_transform(df_combo[column].values.reshape(-1,1))


In [12]:
from sklearn.preprocessing import QuantileTransformer
l1 = [x for x in df_combo.columns.to_list() if x.startswith('player_offensive_ratio_var')]
for column in l1:
    qt5 = QuantileTransformer(output_distribution = 'normal')
    df_combo[column] = qt5.fit_transform(df_combo[column].values.reshape(-1,1))


In [13]:
from sklearn.preprocessing import QuantileTransformer
l1 = [x for x in df_combo.columns.to_list() if x.startswith('team1_offensive_ratio_var')]
for column in l1:
    qt6 = QuantileTransformer(output_distribution = 'normal')
    df_combo[column] = qt6.fit_transform(df_combo[column].values.reshape(-1,1))


In [14]:
from sklearn.preprocessing import QuantileTransformer
l1 = [x for x in df_combo.columns.to_list() if x.startswith('team1_offensive_derived_var')]
for column in l1:
    qt7 = QuantileTransformer(output_distribution = 'normal')
    df_combo[column] = qt7.fit_transform(df_combo[column].values.reshape(-1,1))


In [15]:
from sklearn.preprocessing import QuantileTransformer
l1 = [x for x in df_combo.columns.to_list() if x.startswith('team2_offensive_ratio_var')]
for column in l1:
    qt8 = QuantileTransformer(output_distribution = 'normal')
    df_combo[column] = qt8.fit_transform(df_combo[column].values.reshape(-1,1))


In [16]:
from sklearn.preprocessing import QuantileTransformer
l1 = [x for x in df_combo.columns.to_list() if x.startswith('team2_offensive_derived_var')]
for column in l1:
    qt9 = QuantileTransformer(output_distribution = 'normal')
    df_combo[column] = qt9.fit_transform(df_combo[column].values.reshape(-1,1))


In [17]:
from sklearn.preprocessing import QuantileTransformer
l1 = [x for x in df_combo.columns.to_list() if x.startswith('player_physical_derived_var')]
for column in l1:
    qt9 = QuantileTransformer(output_distribution = 'normal')
    df_combo[column] = qt9.fit_transform(df_combo[column].values.reshape(-1,1))


### Here we have observed that these columns have multimodal kind of distribution, even if they have continuous values.
### So we decided to convert them to object type and then Label Encode them.

In [18]:
l1 = [x for x in df_combo.columns.to_list() if x.startswith('player_general_raw_var')]
for column in l1:
    df_combo[column] = df_combo[column].astype('object')
from sklearn.preprocessing import LabelEncoder
for column in l1:
    lb = LabelEncoder()
    df_combo[column] = lb.fit_transform(df_combo[column].values.reshape(-1,1))
    
    
    
l2 = [x for x in df_combo.columns.to_list() if x.startswith('player_defensive_raw_var')]
for column in l2:
    df_combo[column] = df_combo[column].astype('object')
from sklearn.preprocessing import LabelEncoder
for column in l2:
    lb = LabelEncoder()
    df_combo[column] = lb.fit_transform(df_combo[column].values.reshape(-1,1))
    

l3 = [x for x in df_combo.columns.to_list() if x.startswith('player_offensive_raw_var')]
for column in l3:
    df_combo[column] = df_combo[column].astype('object')
from sklearn.preprocessing import LabelEncoder
for column in l3:
    lb = LabelEncoder()
    df_combo[column] = lb.fit_transform(df_combo[column].values.reshape(-1,1))
    
    
l4 = [x for x in df_combo.columns.to_list() if x.startswith('team1_offensive_raw_var')]
for column in l4:
    df_combo[column] = df_combo[column].astype('object')
from sklearn.preprocessing import LabelEncoder
for column in l4:
    lb = LabelEncoder()
    df_combo[column] = lb.fit_transform(df_combo[column].values.reshape(-1,1))
    
    
l5 = [x for x in df_combo.columns.to_list() if x.startswith('team1_defensive_raw_var')]
for column in l5:
    df_combo[column] = df_combo[column].astype('object')
from sklearn.preprocessing import LabelEncoder
for column in l5:
    lb = LabelEncoder()
    df_combo[column] = lb.fit_transform(df_combo[column].values.reshape(-1,1))
    
    
l6 = [x for x in df_combo.columns.to_list() if x.startswith('team2_offensive_raw_var')]
for column in l6:
    df_combo[column] = df_combo[column].astype('object')
from sklearn.preprocessing import LabelEncoder
for column in l6:
    lb = LabelEncoder()
    df_combo[column] = lb.fit_transform(df_combo[column].values.reshape(-1,1))
    
    
l7 = [x for x in df_combo.columns.to_list() if x.startswith('team2_defensive_raw_var')]
for column in l7:
    df_combo[column] = df_combo[column].astype('object')
from sklearn.preprocessing import LabelEncoder
for column in l7:
    lb = LabelEncoder()
    df_combo[column] = lb.fit_transform(df_combo[column].values.reshape(-1,1))

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### This is just a standard procedure of Label Encoding the inherently 'Object' Type Columns

In [19]:
from sklearn.preprocessing import LabelEncoder
for column in df_combo.select_dtypes('object').columns.to_list():
    lb = LabelEncoder()
    df_combo[column] = lb.fit_transform(df_combo[column].values.reshape(-1,1))
    

### We have selected Categorical Features which we would later pass into models.

In [20]:
cat_feature = ['scout_id','winner','team','team1_system_id','team2_system_id','competitionId','player_position_1','player_position_2']
cat_feature = cat_feature+l1+l2+l3+l4+l5+l6+l7
for column in cat_feature:
    df_combo[column] = df_combo[column].astype('int64')

### Standard Procedure of Doing Train-Test Split

In [21]:
train = df_combo.head(df_train.shape[0])
test = df_combo.tail(df_test.shape[0])

x = train
y = df_train['rating_num']

In [22]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x , y , test_size=0.2 , random_state=101)

# MODELLING

### Checking XGBoost Model Performance

In [23]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

xgb = XGBRegressor(n_estimators = 100)
xgb.fit(x_train,y_train)

preds_valid = xgb.predict(x_test)

print('Training Score',r2_score(y_train,xgb.predict(x_train)))
print()
print('Test Score',r2_score(y_test,xgb.predict(x_test)))

Training Score 0.823472492850303

Test Score 0.30095915125993555


### Checking LGBM Regressor Model Performance

In [24]:
import lightgbm as lgb
from sklearn.metrics import r2_score

lgbr = lgb.LGBMRegressor(n_estimators = 170)
lgbr.fit(x_train,y_train)

preds_valid = lgbr.predict(x_test)

print('Training Score',r2_score(y_train,lgbr.predict(x_train)))
print()
print('Test Score',r2_score(y_test,lgbr.predict(x_test)))

Training Score 0.6841256251840047

Test Score 0.3227835028878834


### Checking CatBoost Model Performance

In [25]:
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score

cbr = CatBoostRegressor(verbose = False , eval_metric = 'R2',cat_features=cat_feature)
cbr.fit(x_train,y_train)

preds_valid = cbr.predict(x_test)

print('Training Score',r2_score(y_train,cbr.predict(x_train)))
print()
print('Test Score',r2_score(y_test,cbr.predict(x_test)))

Training Score 0.7296570243815041

Test Score 0.3672298937536369


### This is the model we finally used, which is a Stacking Regressor of 'CatBoost' and 'LGBM' Regressor

In [26]:
from sklearn.ensemble import StackingRegressor
stk = StackingRegressor([('cbr',CatBoostRegressor(verbose = False , eval_metric = 'R2',cat_features=cat_feature)),
                        ('lgbr',lgb.LGBMRegressor(n_estimators = 170))])
stk.fit(x_train,y_train)

preds_valid = stk.predict(x_test)

print('Training Score',r2_score(y_train,stk.predict(x_train)))
print()
print('Test Score',r2_score(y_test,stk.predict(x_test)))

Training Score 0.7683400181123607

Test Score 0.37708974882960733


# SUBMISSION FILE

### Making the Final Submission File

In [27]:
preds_test = 0.0*cbr.predict(test)+0.0*lgbr.predict(test)+stk.predict(test)
preds_test = preds_test.astype('int64')
preds_test

array([6, 5, 5, ..., 6, 5, 7])

In [28]:
df_sub['rating_num'] = preds_test
df_sub['rating_num'].value_counts()

6     3157
5     2222
7     1949
4      673
8      560
3      100
9       72
2       28
1        8
10       5
Name: rating_num, dtype: int64

In [29]:
df_sub['rating_num'] = preds_test
df_sub.to_csv('submission.csv',index=False)